In [1]:
def combiner(data):
    max_latitude, max_longitude = max([abs(i[0]) for i in data]), max([abs(i[1]) for i in data])
    max_number_bits = len(bin(int(max([max_latitude, max_longitude]) * 10 ** 7))) - 2 + 1
    c = []
    for value in data:
        a, b = value
        a_ve, b_ve = False, False
        bin_a = bin(int(a*10**7))
        bin_b = bin(int(b*10**7))
        if bin_a[0] == '-':
            a_ve = True
            bin_a = bin_a[3:]
        else:
            bin_a = bin_a[2:]
        if bin_b[0] == '-':
            # print ('here')
            b_ve = True
            bin_b = bin_b[3:]
        else:
            bin_b = bin_b[2:]
        difference1 = max_number_bits - len(bin_a)
        difference2 = max_number_bits - len(bin_b)
        if (difference1 > 0):
            if a_ve == True:
                bin_a = '1' + '0'*(difference1-1) + bin_a
            else:
                bin_a = '0'*(difference1) + bin_a
        if (difference2 > 0):
            if b_ve == True:
                bin_b = '1' + '0'*(difference2-1) + bin_b
            else:
                bin_b = '0'*(difference2) + bin_b
        temp = []
        for i in range(len(bin_a)):
            temp.append(bin_a[i])
            temp.append(bin_b[i])
        temp = ''.join(temp)
        c.append(temp)
    return c

def hilbert_curve(c):
    # print c
    two_bit_list = [c[i:i + 2] for i in range(0, len(c), 2)]
    # print two_bit_list
    for i in range(len(two_bit_list)):
        if two_bit_list[i] == '00':
            two_bit_list[i] = '0'
        if two_bit_list[i] == '01':
            two_bit_list[i] = '1'
        if two_bit_list[i] == '10':
            two_bit_list[i] = '3'
        if two_bit_list[i] == '11':
            two_bit_list[i] = '2'
#     print two_bit_list
    a = two_bit_list
    a = ''.join(a)
    #a = '012345321'
#     print a
    if (a.find('0') >= 0):
        a = a[:a.find('0') + 1] + a[a.find('0') + 1:].replace('1', '%temp%').replace('3', '1').replace('%temp%', '3')
    #print a
    if (a.find('3') >= 0):
        a = a[:a.find('3') + 1] + a[a.find('3') + 1:].replace('0', '%temp%').replace('2', '0').replace('%temp%', '2')

    # print a
    new_list = []
    for i in a:
        new_list.append(i)
    # print new_list
    new_list = map(lambda x: bin(int(x)), new_list)
    new_list = map(lambda x: x[2:], new_list)
    # print new_list
    maxi = 0
    for i in new_list:

        if (len(i) > maxi):
            maxi = len(i)

    # print maxi

    for i in range(len(new_list)):
        if (len(new_list[i]) < maxi):
            difference = maxi - len(new_list[i])
            new_list[i] = '0' * difference + new_list[i]

    new_list = ''.join(new_list)
    new_list=int(new_list,2)
    #print new_list
    return new_list

import pandas as pd
import sqlite3

# Read sqlite query results into a pandas DataFrame
con = sqlite3.connect("data/locations.db")
df = pd.read_sql_query("SELECT * from gpsdata", con)

# verify that result of SQL query is stored in the dataframe
print(df.head())

con.close()

latlon = df[['Latitude', 'Longitude']]
data = latlon.as_matrix()
data = data.tolist()

z = combiner(data)

h = []
for i in z:
    h.append(hilbert_curve(i))

   index             DateTime   Latitude  Longitude
0      0  2017-08-06 16:31:09  32.733327 -97.113758
1      1  2017-08-06 16:20:07  32.733336 -97.113651
2      2  2017-08-06 16:16:11  32.733144 -97.112573
3      3  2017-08-06 16:13:12  32.733064 -97.112572
4      4  2017-08-06 16:12:09  32.733649 -97.105925


In [3]:
datetime = df['DateTime']
Latitude = df['Latitude']
Longitude = df['Longitude']

In [4]:
df = pd.DataFrame({'Latitude':Latitude, 'Longitude':Longitude,
                   'DateTime':datetime, 'Z':z, 'H':h})

df['DateTime'] = pd.to_datetime(df['DateTime'])

conn=sqlite3.connect('data/locations.db')
df.to_sql(name='gpsdata', con=conn, if_exists='replace')

In [5]:
len(h)

84814